In [1]:
import pandas as pd

In [2]:
file_path = r"E:\AI_HUB\data\Training\labeled_data\TL_csv\cleaned_data\tn_visit_area_info_v2_cleaned_WR.csv"

tn_visit_area_info = pd.read_csv(file_path)

tn_visit_area_info

,VISIT_AREA_ID,TRAVEL_ID,VISIT_AREA_NM,ROAD_NM_ADDR,X_COORD,Y_COORD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,VISIT_SEASON,W,WR
0,2308280004,g_g005682,피즈,제주특별자치도 제주시 애월읍 애월로 29,126.311823,33.461553,5,5,5,Summer,5.0,4.351057
1,2308280005,g_g005682,명월 국민학교,제주특별자치도 제주시 한림읍 명월로 48,126.264872,33.388738,5,5,5,Summer,5.0,4.397970
2,2308280006,g_g005682,연리지 가든,제주특별자치도 제주시 한경면 두조로 190-20,126.202321,33.345142,5,3,4,Summer,4.1,4.209066
3,2308280007,g_g005682,아르떼 뮤지엄 제주,제주특별자치도 제주시 애월읍 어림비로 478,126.345011,33.396701,5,3,5,Summer,4.4,4.161649
4,2308290002,g_g005682,둘레길 중문 본점,제주특별자치도 서귀포시 천제연로 209-1,126.427123,33.251722,5,5,5,Summer,5.0,4.182932
...,...,...,...,...,...,...,...,...,...,...,...,...
16689,2309230003,h_h006872,항파두리 항몽 유적지,제주특별자치도 제주시 애월읍 항파두리로 50,126.407923,33.452748,4,4,4,Fall,4.0,4.162115
16690,2309230004,h_h006872,소길별하,제주특별자치도 제주시 애월읍 소길남길 34-37,126.375776,33.429839,4,4,4,Fall,4.0,4.324231
16691,2309220002,h_h006874,꼬스뗀뇨,제주특별자치도 제주시 구좌읍 해맞이해안로 2080,126.905244,33.509868,3,2,2,Fall,2.4,4.461676
16692,2309220003,h_h006874,마이 피기 팬트리,제주특별자치도 제주시 구좌읍 하도13길 6,126.888132,33.508700,4,4,4,Fall,4.0,4.176346


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
# 협업 필터링에 필요한 열만 선택
cf_data = tn_visit_area_info[["TRAVEL_ID", "VISIT_AREA_ID", "W"]]

# 사용자-아이템 행렬 생성 (Pivot Table)
rating_matrix = cf_data.pivot_table(index='TRAVEL_ID', columns='VISIT_AREA_ID', values='W', fill_value=0)

In [9]:
# 관광지 간 유사도 행렬 계산
item_similarity = cosine_similarity(rating_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, index=rating_matrix.columns, columns=rating_matrix.columns)

In [11]:
# 사용자 기반 관광지 추천 함수
def recommend_for_user(travel_id, top_n=5):
    if travel_id not in rating_matrix.index:
        return f"사용자 ID {travel_id}가 데이터에 없습니다."

    # 사용자가 방문한 관광지 목록 가져오기
    visited_places = rating_matrix.loc[travel_id]
    visited_places = visited_places[visited_places > 0].index  # 평가한 관광지만 선택

    # 방문한 관광지와 유사한 관광지 찾기
    similar_places = item_similarity_df[visited_places].mean(axis=1).sort_values(ascending=False)

    # 사용자가 이미 방문한 관광지는 제외
    similar_places = similar_places.drop(visited_places, errors='ignore')

    # 관광지 정보와 WR을 결합하여 추천 리스트 생성
    recommended_places = tn_visit_area_info[tn_visit_area_info["VISIT_AREA_ID"].isin(similar_places.index)]
    recommended_places = recommended_places.groupby(["VISIT_AREA_ID", "VISIT_AREA_NM"]).agg({"WR": "mean"}).reset_index()
    recommended_places = recommended_places.sort_values(by="WR", ascending=False).head(top_n)

    return recommended_places

In [13]:
recommend_for_user(2308290003)

'사용자 ID 2308290003가 데이터에 없습니다.'